In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Train Model với Data
- Trong bước này, model được train trực tiếp từ tập train mà chưa qua bước xử lí nào.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
# import os
import os
import cv2
import glob
import numpy as np
def getYourFruits(fruits, data_type, print_n=False, k_fold=False):
    images = []
    labels = []
    val = ['training', 'testing']
    if not k_fold:
        path = "/content/gdrive/MyDrive/AIL_project/traffic/Final Dataset/" + data_type + "/"
        for i,f in enumerate(fruits):
            p = path + f
            j=0
            for image_path in glob.glob(os.path.join(p, "*.jpg")):
                image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                image = cv2.resize(image, (100, 100))
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                images.append(image)
                labels.append(i)
                j+=1
            if(print_n):
                print("There are " , j , " " , data_type.upper(), " images of " , fruits[i].upper())
        images = np.array(images)
        labels = np.array(labels)
        return images, labels
    else:
        for v in val:
            path = "/content/gdrive/MyDrive/AIL_project/traffic/Final Dataset/" + v + "/"
            for i,f in enumerate(fruits):
                p = path + f
                j=0
                for image_path in glob.glob(os.path.join(p, "*.jpg")):
                    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
                    image = cv2.resize(image, (100, 100))
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    images.append(image)
                    labels.append(i)
                    j+=1
        images = np.array(images)
        labels = np.array(labels)
        return images, labels

def getAllFruits():
    fruits = []
    for fruit_path in glob.glob("/content/gdrive/MyDrive/AIL_project/traffic/Final Dataset/training/*"):
        fruit = fruit_path.split("/")[-1]
        fruits.append(fruit)
    return fruits
from sklearn.preprocessing import StandardScaler
fruits = ['Empty', 'Low' , 'Medium', 'High', 'Traffic Jam']


#Get Images and Labels
X, y =  getYourFruits(fruits, 'training')
X_test, y_test = getYourFruits(fruits, 'testing')

#Scale Data Images
scaler = StandardScaler()
X_train = scaler.fit_transform([i.flatten() for i in X])
X_test = scaler.fit_transform([i.flatten() for i in X_test])

**SVM**

In [4]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', gamma = 'auto')
clf.fit(X_train, y)

SVC(gamma='auto', kernel='linear')

In [5]:
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

In [6]:
accuracy

0.659375

**KNN**

In [7]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=4)
model_knn.fit(X_train, y)
y_pred_knn = model_knn.predict(X_test)

In [8]:
accuracy_knn = accuracy_score(y_test, y_pred_knn)
accuracy_knn

0.425

- Các model bên trên có chỉ số accuracy khá thấp.
- Hiệu suất của mô hình có thể bị ảnh hưởng bởi các tham số truyền vào khi huấn luyện mô hình. Vậy để tối ưu hóa các tham số để mô hình có hiệu suất tốt, ta có thể sử dụng phương pháp Hyperparameter Tuning ( điều chỉnh siêu tham số) bằng cách:
 + **Train model bằng nhiều bộ parameters rồi chọn ra bộ parameter có hiệu suất cao nhất**

**Ví dụ với thuật toán KNN**

In [20]:
# KNN
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [21]:
scaler= StandardScaler()
knn = KNeighborsClassifier()

In [22]:
temp = [('scaler', scaler),('knn', knn)]

In [23]:
pipeline = Pipeline(temp)

In [24]:
from sklearn.model_selection import GridSearchCV
param_grid = {'knn__n_neighbors':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19,
                                              20, 21, 22, 23, 24, 25, 26, 27,
                                              28, 29]}
grid_search = GridSearchCV(cv=5,estimator=pipeline, param_grid=param_grid, scoring='accuracy')
grid_search.fit(X_train, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19,
                                              20, 21, 22, 23, 24, 25, 26, 27,
                                              28, 29]},
             scoring='accuracy')

In [25]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'knn__n_neighbors': 13}


In [30]:
model_knn_ex = KNeighborsClassifier(n_neighbors=13)
model_knn_ex.fit(X_train, y)

KNeighborsClassifier(n_neighbors=13)

In [31]:
from sklearn.metrics import accuracy_score

y_pred_knn = model_knn_ex.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_knn)
accuracy

0.4625

**Trong mô hình KNN, n_neighbors là một siêu tham số quyết định số K lân cận. Để tối ưu hóa hiệu xuất của mô hình, thử nghiệm nhiều giá trị K khác nhau, thông qua điều chỉnh siêu tham số.**
- Ở trường hợp ví dụ trên, hiệu xuất của mô hình đã được cải thiện sau khi chọn được n_neighbors hợp lí, nhưng nó không cải thiện vượt trội. Vì những lí do sau:
+ Data đang bị không cân bằng
+ Và ảnh đang còn rất lớn, nhiều outliers.

**Conclusion:**
- Sau khi train model cho kết quả khá thấp, ví dụ như thuật toán KNN.
- Lí do model có accurancy thấp là do các parameter chưa tối ưu, bên cạnh đó, data đang bị imbalanced.
- Để tìm ra các parameter phù hợp thì ta sẽ train model với một bộ những parameter. Và chọn ra một bộ parameter có hiệu suất cao nhất.
- Để giải quyết vấn đề dữ liệu mất cân bằng, sử dụng thuật toán SMOTE.
- Nhưng có một vấn đề nữa ở đây đó là thời gian train model rất lâu (đối với trường hợp chạy nhiều bộ parameters), để giải quyết vấn đề này ta sẽ dùng PCA để giảm chiều dữ liệu ( nén dữ liệu), giúp tăng hiệu suất của model và tránh overfitting. ( cách này sẽ được trình bày ở file sau).

------
# **END**